In [ ]:
import os
import requests
from datetime import datetime
import pandas as pd 
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)


In [ ]:
# Base endpoints
MAIN_URL      = "https://epic.gsfc.nasa.gov/api/natural"
MAIN_PATH     = "D:/Data_Data_no_Mi/Astronomics_island/NASA/downloaded"
os.chdir(MAIN_PATH)

In [ ]:


def get_available_dates():
    """Returns a list of strings like '2025-07-01', etc."""
    resp = requests.get(MAIN_URL + "/all")
    resp.raise_for_status()
    return resp.json()

def get_metadata_for_date(date_str):
    """Fetches the list of image metadata for a particular date."""
    # date_str must be 'YYYY-MM-DD'
    MAIN_URL_MODIFIED = MAIN_URL.replace('natural','enhanced')
    resp = requests.get(f"{MAIN_URL_MODIFIED}/date/{date_str}")
    resp.raise_for_status()
    return resp.json()

def format_request(list_identifiers):
    d = {'identifier': list_identifiers}
    df = pd.DataFrame(data=d)
    print(df.shape)
    df = df.drop_duplicates()
    print(df.shape)

    df['year'] = df['identifier'].apply(lambda x: x[0:4])
    df['month'] = df['identifier'].apply(lambda x: x[4:6])
    df['day'] = df['identifier'].apply(lambda x: x[6:8])
    df['request'] = 'https://epic.gsfc.nasa.gov/archive/natural/'+df['year']+'/'+df['month']+'/'+df['day']+'/png/epic_1b_'+df['identifier']+'.png'
    return df 

def download_image(df):
        for i in range(0,len(df)+1):
            try: 
                #print(df['request'].iloc[i])
                picture_request = requests.get(f'{df['request'].iloc[i]}')  
                if picture_request.status_code == 200:
                    with open(f"{df['identifier'].iloc[i]}.png", 'wb') as img:  
                        img.write(picture_request.content)
            except Exception as e: 
                print('---------------------------------------')
                print(f'Error in url: {df['request'].iloc[i]}')
                print(f'Error description: {e}')
                pass 

In [ ]:


if __name__ == "__main__":
    
    # Paso 1: Obtener todas las fechas 
    dates = get_available_dates()
    list_identifiers = []
    
    # Paso 2: Obtener todos los identificadores
    for i in dates:
        date_view = i['date']
        meta = get_metadata_for_date(date_view)
        for m in meta:
            identifier = m["identifier"]
            list_identifiers.append(identifier)

    # Paso 3: Request de imagen y descargar
    df = format_request(list_identifiers)
    try: 
        df.to_excel('D:/Data_Data_no_Mi/Astronomics_island/NASA/Images_NASA_earth_2015_25.xlsx', index = False)
    except: 
        print("No se pudo exportar el excel")
    download_image(df)

    
